<a href="https://colab.research.google.com/github/AnirudhArrepu/Network-Anamoly-Detection-Model/blob/main/NetworkAnamoly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from google.colab import drive
import numpy as np

drive.mount('/content/drive', force_remount=True)

df = pd.read_csv("/content/drive/MyDrive/KDDCup99.csv")
df=pd.DataFrame(df)
df.describe()

Mounted at /content/drive


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,lnum_compromised,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
count,494020.000000,4.940200e+05,4.940200e+05,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,...,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000
mean,47.979400,3.025616e+03,8.685308e+02,0.000045,0.006433,0.000014,0.034519,0.000152,0.148245,0.010212,...,232.471248,188.666052,0.753781,0.030906,0.601936,0.006684,0.176754,0.176443,0.058118,0.057412
std,707.747185,9.882191e+05,3.304003e+04,0.006673,0.134805,0.005510,0.782103,0.015520,0.355343,1.798328,...,64.744601,106.040205,0.410780,0.109259,0.481309,0.042133,0.380593,0.380920,0.230590,0.230141
min,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,4.500000e+01,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,46.000000,0.410000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,5.200000e+02,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,255.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,1.032000e+03,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,255.000000,1.000000,0.040000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,58329.000000,6.933756e+08,5.155468e+06,1.000000,3.000000,3.000000,30.000000,5.000000,1.000000,884.000000,...,255.000000,255.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [3]:
features = ["duration", "src_bytes", "protocol_type","service","flag","hot","count", "srv_serror_rate", "srv_rerror_rate"]
X = df[features]
y = df["label"]
def labelconv(label):
  if label == 'normal':
    return 0
  else:
    return 1
y=y.apply(labelconv)


In [4]:
label_encoders = {}
X_encoded = X.copy()
for col in X.columns:
  if X[col].dtype == 'object':
    label_encoders[col] = LabelEncoder()
    X_encoded[col] = label_encoders[col].fit_transform(X[col])

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded,y, test_size=0.2, random_state=10)
contamination_values=[0.1, 0.2, 0.3]
best_model = None
best_f1_score = 0
for values in contamination_values:
  model = IsolationForest(contamination=values)
  model.fit(X_train)
  anamoly_scores=model.decision_function(X_test)
  y_pred = anamoly_scores > 0
  report = classification_report(y_test,y_pred,labels=[0 ,1], target_names=[0,1], output_dict=True)
  f1_score=report[1]['f1-score']
  print(f1_score)
  if f1_score >= best_f1_score:
    best_f1_score=f1_score
    best_model=model
if best_model:
  anomaly_scores = best_model.decision_function(X_test)
  anomalies = X_test[anomaly_scores > 0]
  print(anomalies)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


0.8634703467967403


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


0.8465733514487029


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


0.7753772429296396
        duration  src_bytes  protocol_type  service  flag  hot  count  \
73049          0          0              1       45     5    0    241   
5763           0        163              1       22     9    0      1   
62250          0          0              1       45     5    0    183   
166661         0       1032              0       14     9    0    511   
483637         0        321              1       22     9    0      4   
...          ...        ...            ...      ...   ...  ...    ...   
380595         0          0              1       45     5    0    272   
337501         0       1032              0       14     9    0    511   
387246         0          0              1       45     5    0    220   
399613         0        520              0       14     9    0    452   
429453         0        520              0       14     9    0    511   

        srv_serror_rate  srv_rerror_rate  
73049               1.0              0.0  
5763              